In [1]:
!apt update && apt install -y openslide-tools
!pip install openslide-python

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:13 http://security.ubuntu.com/ubu

In [2]:
!rm -r "/content/drive/MyDrive/Teaching&Thesis/Teaching_dataset/teaching-MLinAPP"
!git clone https://github.com/frpnz/teaching-MLinAPP.git "/content/drive/MyDrive/Teaching&Thesis/Teaching_dataset/teaching-MLinAPP"

Cloning into '/content/drive/MyDrive/Teaching&Thesis/Teaching_dataset/teaching-MLinAPP'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 69 (delta 30), reused 54 (delta 15), pack-reused 0
Unpacking objects: 100% (69/69), done.


In [3]:
import os
import sys
import openslide
import numpy as np
from glob import glob
import tensorflow as tf
import matplotlib.pyplot as plt
import openslide.deepzoom as dz
ROOTDIR_WSI = "/content/drive/MyDrive/Teaching&Thesis/Teaching_dataset/CRC_ROIs_2_classes"
ROOTDIR_DATA = "/content/drive/MyDrive/Teaching&Thesis/Teaching_dataset/"
ROOTDIR_SRC = "/content/drive/MyDrive/Teaching&Thesis/Teaching_dataset/teaching-MLinAPP/src"
sys.path.append(ROOTDIR_DATA)
sys.path.append(ROOTDIR_SRC)
tileSize = 500
overlap = 6
input_shape = (512, 512)

In [4]:
from resnet import ResNet

In [5]:
files = glob(os.path.join(ROOTDIR_WSI, '*.svs'))
openSlideObjects = [(openslide.OpenSlide(slide), os.path.basename(slide).split('_')[1]) for slide in files]
deepZoomObjects = [(dz.DeepZoomGenerator(slide, tile_size=tileSize, overlap=overlap, limit_bounds=True),label) for slide, label in openSlideObjects]
deepZoomObjects = [(slide, label, slide.get_tile(10,(0,0))) for slide, label in deepZoomObjects]

In [6]:
len(deepZoomObjects)

17

In [7]:
from math import floor

classDict = {
    'AC':0,
    'H':1,
}

def createTilePlaceholders(deepZoomObject,label,level,preview):
    if level>=deepZoomObject.level_count:
        raise(RuntimeError('Requested level is not available'))

    out=[]
    preview = tf.reshape(tf.cast(preview.getdata(),dtype=tf.uint8),(preview.size[0],preview.size[1],3))
    preview = tf.image.convert_image_dtype(preview,dtype=tf.float32)
    previewScale = 2**(level-10)

    for i in range(deepZoomObject.level_tiles[level][0]):
        for j in range(deepZoomObject.level_tiles[level][1]):
            tmp={
                'deepZoomObject':deepZoomObject,
                'level':level,
                'coordinates':(i,j),
                'label':label}
            position = (floor(i*tileSize/previewScale),floor(j*tileSize/previewScale),
                        floor(i + 1*tileSize/previewScale), floor(j + 1*tileSize/previewScale))
            crop = preview[position[0]:position[1],position[2]:position[3]]
            if tf.reduce_mean(tf.math.reduce_std(crop,axis=-1))>0.02:
                out.append(tmp)
    return out

tilePlaceholders = [createTilePlaceholders(slide,
                                           label,
                                           slide.level_count-2,
                                           preview) for slide, label, preview in deepZoomObjects]
tilePlaceholders = [item for sublist in tilePlaceholders for item in sublist]

def toImage(x):
    tile = tilePlaceholders[x.numpy()]
    PILObject = tile['deepZoomObject'].get_tile(tile['level'], tile['coordinates'])
    imSize = PILObject.size
    img = tf.reshape(PILObject.getdata(), (imSize[0], imSize[1], 3))
    return tf.image.convert_image_dtype(img, dtype=tf.float32), tf.cast(tf.one_hot(classDict[tile['label']], 2, name='label', axis=-1), tf.float32)

def filterWhite(x, label):
    if tf.reduce_mean(tf.math.reduce_std(x,axis=-1)) < 0.02:
        #tf.print('Skipped (white)')
        return False
    return True

def filterBorder(x):
    if x.shape[0] != tileSize + 2*overlap or x.shape[1] != tileSize + 2*overlap:
        #tf.print('Skipped (border) %d - %d'%(x.shape[0],x.shape[1]))
        return False
    else:
        return True

def _fixup_shape(image, label):
    image.set_shape([512, 512, 3])
    label.set_shape([]) # I have 19 classes
    # weights.set_shape([None])
    return image, label

dataset = tf.data.Dataset.from_tensor_slices([i for i in range(len(tilePlaceholders))])
dataset = dataset.shuffle(50000)
dataset = dataset.map(lambda x: tf.py_function(toImage, [x], Tout=[tf.float32, tf.float32]), num_parallel_calls=8)
dataset = dataset.filter(filterWhite)
dataset = dataset.filter(lambda x, label: tf.py_function(filterBorder, [x], tf.bool))
dataset = dataset.map(_fixup_shape)
dataset = dataset.batch(32)
dataset = dataset.prefetch(buffer_size=1)

In [ ]:
for x, y in dataset.take(1):
    print(x.shape)
    print(y.shape)

In [ ]:
inv_classDict = {v: k for k, v in classDict.items()}
for batch_x, batch_y in dataset.take(2):
    fig, ax = plt.subplots(5, 5, figsize=(18, 18))
    ax = ax.ravel()
    j = 0
    for image, label in zip(batch_x[:25], batch_y[:25]):
        label = label.numpy()
        img = image.numpy()
        ax[j].imshow(img)
        ax[j].axis('off')
        ax[j].set_title("Class: {}".format(inv_classDict[int(np.argmax(label))]))
        j += 1

In [ ]:
# def process_data(image, label, num_classes):
#   return tf.cast(image, tf.float32) / 255., tf.one_hot(tf.cast(label, tf.int32), num_classes, name='label', axis=-1)

# dataset = dataset.map(lambda x, y: process_data(x, y, 2))
data = iter(dataset)

In [ ]:
inputs = tf.keras.Input((512, 512, 3))
x = tf.keras.applications.ResNet50(include_top=False, weights="imagenet")(inputs)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(2, activation="softmax")(x)

In [ ]:
model = tf.keras.models.Model(inputs=inputs, outputs=x)

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
loss = tf.keras.losses.categorical_crossentropy
model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])

In [ ]:
model.fit(dataset)